<img src="files/Pics/LOGOS.png" width="800">

# Labor 07 - Neurális hálózatok tanítása

### Kézzel írott számok III.
Ebben a feladatban tanítani fogjuk a neurális hálózatunkat, hogy az képes legyen felismerni kézzel írott számjegyeket 0-9-ig.

### 1: Adatok beolvasása 
Importáljuk be a használni kívánt csomagokt és olvassuk be az adatokat

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat

data = loadmat("Lab7data.mat")                      # adatok beolvasása
X = data["X"]
y = data ["y"]                                      # adatok elrendezése
del data
m = X.shape[0]
print('Shape of X and y in order:')
print(X.shape)
print(y.shape)

data = loadmat("Lab7Weights.mat")                   # súlyok beolvasása
w1 = np.array(data["Theta1"])
w2 = np.array(data["Theta2"])                       # súlyok elrendezése
print('Shape of w1 and w2 in order:')
print(w1.shape)
print(w2.shape)
del data

Amit érdemes megfigyelni a kezdeti adatainkon:
- A minták egy 5000x400-as mátrixba vannak rendezve, ami azt jenelnti, hogy az 5000 darab 20x20 pixeles képünk van. A képeket kiterítve tároljuk a mátrixban.
- Kimenetként egy számot várunk, melyek egy 5000x1-es vektorban vannak eltárolva.
- A háló működéséhez szükséges súlymátrixok rendre 25x401 és 10x26. Ez azt jelenti, hogy a 400 kezdeti bemeneti paramáterünk ki van egészítve a BIAS taggal és a rejtett rétegben 25 neuron található.
- A rejetett réteghez is csatolva van a BIAS tag és így teremti meg a kapcsolatot a második súlyvektor a 26 rejtett neuton és a 10 lehetséges kimenet között.

A háló működését tekintve egy elem 10 osztályba tartozási valószínűségét fogjuk vizsgálni, melyek közül a legnagyobbat választva (MaxPooling) fogjuk meghatározni a végső számot.

### 2: Adatok vizualizálása

In [ ]:
print("Printing some random data ...")                              
fig, ax = plt.subplots(10,10, figsize =(8,8))                  # subplots - 10x10 8x8as képnagysággal
for i in range(10):
    for j in range(10):
        ax[i,j].imshow(X[np.random.randint(0,m+1),:].reshape(20,20, order = "F"), cmap="hot")
        ax[i,j].axis("off")
plt.show()

### 3. Elméleti áttekintés

A könyebb megértés érdekében tekintsük az alábbi egyszerűsített példát. 

<img src="files/Pics/L07_Network.png">

A felvázolt neurális háló 3 rétegől áll.
- Bemeneti réteg, mely 2 bemeneti neuront tartalmaz plusz a hozzáadott BIAS tagot.
- Rejtett réteg, mely 3 neuront tartalmaz plusz a hozzárendelt BIAS tagot.
- Kimeneti réteg, mely 3 neuront tartalmaz, melyek közül a maximumot kiválasztva kapjuk meg a tényleges kimenetet.

A neuronok szerepét tekintve a BIAS neuronok kezelése kicsit eltérő. Mivel a BIAS nem változó, hanem egy konstans 1-es érték, hogy a hozzá tartozó súly egy független paraméter lehessen. Ennek megfelelően csak a súlyokkal való beszorzásnál van szerepe és minden réteghez igény szerint hozzá rendelhető. 

Jelöléseket tekintve vezessük be az alábbiakat: <br>
$a^{(j)}_{i}$ - j. réteg i. neuronjának aktivációja <br>
$s^{(j)}_{i}$ - j. réteg i. neuronjának szumma összege <br>
$w^{(j)}_{lk}$ - j. réteg l. neuronjából j+1. rltek k. neuronjába mutató súly (weight) <br>
$x^{(m)}_{n}$ - az m. bemeneti minta n. változója. ($x^{(1)}_0 = 1)$ a BIAS tag.<br> 
$\hat y$ - a kimenet

A forward lépést tehát az alábbiak szerint építhetjük fel: <br>
$x^{(1)}$-et kiegészítjük a BIAS taggal és beszorozzuk az első súlymátrixszal.

$ \underset{1\times 3}{\mathrm{x^{(1)}}} \times \underset{3\times 3}{\mathrm{w^{(1)}}} = \underset{1\times 3}{\mathrm{s^{(2)}}} $

Elvégezzük a rejtett réteg neuronjaiban az aktivációt. Aktivációs függvénynek használjuk a szigmoid függvényt.

$ \underset{\color{red}{1\times 3}}{\mathrm{a^{(2)}}} = f(\underset{1\times 3}{\mathrm{s^{(2)}}}) = sigmoid(\underset{1\times 3}{\mathrm{s^{(2)}}}) $ 

Hozzá rendeljük a BIAS tagot a rejtett réteghez az aktiváció után, de még a súly beszorzás előtt!

$ \underset{\color{red}{1\times 4}}{\mathrm{a^{(2)}}} \times \underset{4\times 3}{\mathrm{w^{(2)}}} = \underset{1\times 3}{\mathrm{s^{(3)}}}  $

Az aktivációt követően megkapjuk a kimeneti réteg neuronjoinak értékét. 

$ \underset{1\times 3}{\mathrm{a^{(3)}}} = f(\underset{1\times 3}{\mathrm{s^{(3)}}}) = sigmoid(\underset{1\times 3}{\mathrm{s^{(3)}}}) $

A kimeneti réteg a predikciónkat tartalmazza. 

$ \underset{1\times 3}{\mathrm{\hat{y}}} = \underset{1\times 3}{\mathrm{a^{(3)}}} $

Klasszifikációs probléma esetén a maximumot, vagyis a legvalószínűbb csoportot kiválasztva kaphatjuk meg a végső predikciót. 

#### Költség függvény

Költség függvénynek használjuk az eddig bevált MSE (Mean Square Error) módszert. Nézzük meg egy kimeneti neuron esetében hogyan alakul a hiba számítás. A BIASokat is figyelmen kívül hagyva koncentráljunk csak az ábrán sötét színnel ábrázolt kapcsolatokra. <br>

$ C = \sum\{\frac{1}{2}(y-\color{red}{\hat y})^2 \} $

$ C = \sum\{\frac{1}{2}(y-{\color{red}{a^{(3)}}})^2 \} $

$ C = \sum\{\frac{1}{2}(y-f({\color{red}{s^{(3)}}}))^2\} $

$ C = \sum\{\frac{1}{2}(y-f({\color{red}{a^{(2)}}}w^{(2)}))^2\} $

$ C = \sum\{\frac{1}{2}(y-f(f({\color{red}{s^{(2)}}})w^{(2)}))^2\}$

$ C = \sum\{\frac{1}{2}(y-f(f(xw^{(1)})w^{(2)}))^2\} $

A hiba számítás tehát visszavezethető a bemeneti változók és a hálóban szereplő súlyok függvényére. A formula tetszőleges számú rétegre alkalmazható.

#### Hiba visszaterjesztés (Back Propagation)
A hiba visszaterjesztés során a kimeneti rétegtől kezdve visszafelé kiszámoljuk, hogy az egyes neuronok mekkora mértékben felelősek a hiba kialakulásáért. Gyakorlatilag a költség függvény kiszámítása során használt lépéseket vissza fele kell elvégeznünk, ami azt jelenti szükségünk lesz az aktivációs függvény deriváltjára. <br>
##### Szigmoid függvény és deriváltja
$g(z) = \frac{1}{1+e^{-z}}$ <br>
$
\begin{split}
g'(z) = 
& = \frac{d}{dz}\frac{1}{1+e^{-z}} \\
& = \frac{1}{(1+e^{-z})^2}(e^{(-z)}) \\
& = \frac{1}{1+e^{-z}}(1-\frac{1}{(1+e^{-z})}) \\
& = g(z)(1-g(z))
\end{split}
$
<img src="files/Pics/L07_SigmoidDeriv.png" width=400>

A hiba visszaterjesztéshes (Back propagation) a költség függvényt a megfelelő súlymátrix elemei szerint parciálisan deriválva kaphatjuk meg a hiba mértékét a rejtett réteg neuronjaiban. 

$ \begin{split}
\frac{\partial C}{\partial w^{(2)}} = \frac{\partial \sum \frac{1}{2}(y-\hat{y})^2}{\partial w^{(2)}} = \sum (\frac{\partial \frac{1}{2}(y-\hat{y})^2}{\partial w^{(2)}}) 
\end{split}$

A könnyebb átláthatóság miatt vezessük le egy elemre a deriválást. <br>

$
\begin{split}
\frac{\partial \frac{1}{2}(y-\hat{y})^2}{\partial w^{(2)}} 
& = (y-\hat{y})(-\frac{\hat{y}}{\partial w^{(2)}}) \\
& = -(y-\hat{y}) \cdot \frac{\partial \hat{y}}{\partial s^{(3)}}\cdot \frac{\partial 	s^{(3)}}{\partial w^{(2)}}\\	
& = \color{red}{-(y-\hat{y}) \cdot f'(s^{(3)})}\cdot
\frac{\partial a^{(2)}w^{(2)}}{\partial w^{(2)}}\\
& = {\color{red} {\delta^{(3)}}\cdot a^{(2)}}
\end{split}
$<br>

Vezesük be $ \delta^{(j)}_{i} $, mint a j. réteg i. neuronjához rendelt hibatagot. <br>
Mátrixos formában kibővítve a dimenziók ellenőrzése után a következő összefüggést kapjuk: <br>
$ (a^{(2)})^T \delta^{(3)} $


#### Tanítás
A tanító fázis során a korábban megismert módon a súlyokat a Gradiens Descent módszer segítségével módosítjuk. A súlymódosításhoz definiálnunk kell egy tanulási rátát $(\mu)$ és opcionálisan a büntető tagot is megadhatjuk. <br>

$ w^{(1)} = w^{(1)} - \mu \frac{\partial C}{w^{(1)}}+ regularization $

$ w^{(2)} = w^{(2)} - \mu \frac{\partial C}{w^{(2)}}+ regularization $

#### Algoritmus lépései
Foglaljuk össze az algoritmus megvalósítandó lépéseit

1, $\hspace{2mm} xw^{(1)} = s^{(2)} $

2, $\hspace{14mm} f(s^{(2)}) = a^{(2)}$ 

3, $\hspace{32mm} a^{(2)} w^{(2)} = s^{(3)} $

4, $\hspace{48mm} f(s^{(3)}) = \hat{y} $

5, $\hspace{2mm} C = \sum\{\frac{1}{2}(y-\hat y)^2 \} $

6, $\hspace{2mm} -(y-\hat{y}) \cdot f'(s^{(3)}) = \delta^{(3)} $

7, $\hspace{28mm} (a^{(2)})^T \delta^{(3)} =  \frac{\partial C}{\partial w^{(2)}} $

8, $\hspace{39mm} \delta ^{(3)} \cdot (w^{(2)})^T \cdot f'(s^{(2)})= \delta ^{(2)} $

9, $\hspace{78mm} x^T \delta^{(2)} =  \frac{\partial C}{\partial w^{(1)}} $

10, $\hspace{2mm} w^{(1)} = w^{(1)} - \mu \frac{\partial C}{w^{(1)}}+ regularization $

$\hspace{8mm}w^{(2)} = w^{(2)} - \mu \frac{\partial C}{w^{(2)}}+ regularization $

### 3: Paraméterek előkészítése

In [ ]:
Lambda = 1                                              # Büntetés mértéke
input_layer_size = 400                                  # bemeneti réteg nagysága
hidden_layer_size = 25                                  # rejtett réteg nagysága
num_labels = 10                                         # labelek száma
nn_params = np.append(w1.flatten(), w2.flatten())       # kialapítjuk a w1 és w2-t és egymás után füzzük

Az architecktúra nyomonkövetése átláthatóbb, ha az egyes rétegek neuronszámait tároljuk a BIAS tagok nélkül. A súlyokat pedig kiterítve egy változóban kezeljük. A mátrixok méretei az egyes rétegek neuron számai alapján visszaállíthatóak. Továbbá szükségünk lesz a tanulási rátára, mint előre beállított paraméterre.

### 4: Költségfüggvény
A költség függvény implementálása előtt szükségünk lesz egy aktivációs függvényre és annak deriváltjára. Aktivációs függvénynek ismét a szigmoid függvényt választjuk.

In [ ]:
def sigmoid(z):
##########################################    
   
    
##########################################    
    return g

def sigmoidGradient(z):
##########################################    
   
    
##########################################    
    return g_deriv

A költség függvény paraméterként meg fogja kapni a súly paramétereket, a háló rétegméreteit, illetve a mintákat, a mintákhoz tartozó cimkéket és a tanulási rátát. Mivel a súlyparamétereket egy változóban adjuk át a függvénynek, így a függvényen belül kell visszaállítanunk a megfelelő mátrix dimenziókat a súlymátrixoknak.

##### One Hot Encoding

A neurális háló kimenetihez igazításhoz szükségünk lesz még egy kis átalakításra. A minták címkéit egy számmal adtuk meg, viszont egy 10 csoportos klasszifikációs problémán dolgozunk. Ennek megfelelően 10 db kimeneti neuronunk lesz, amik azt fogják megadni, hogy mekkora a valószínűsége annak, hogy az adott minta adott osztályba tartozik. A legnagyobb valószínűség alapjén fogjuk tehát meghatározni a prediktált osztályt. Ehhez az átalakításhoz pedig a One Hot Encoding módszert (csak egy 1-es) fogjuk használni. A módszer lényege, hogy minden címkét átalakítunk egy olyan vektorrá, aminek elemeinek száma megegyezik az osztályok számával és csak egy db 1-est tartalmaz a megfelelő osztály oszlopába. 

Eredeti címkéink a számok 0-9-ig (bal oldalt), a hozzájuk tartozó átalakított címkéink vektorai (jobb oldalt). Jegyezzük meg, hogy az átalakított címkék vektorait összerendezve nagyon hasonlít egy egység mátrixra.

<img src="files/Pics/L07_OneHot.png" width=300>

In [ ]:
def nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, Lambda):
    # nn_params visszaalakítása w1 és w2 mátriokká
    w1 = nn_params[:((input_layer_size + 1) * hidden_layer_size)].reshape(hidden_layer_size, input_layer_size + 1)
    w2 = nn_params[((input_layer_size + 1) * hidden_layer_size):].reshape(num_labels, hidden_layer_size + 1)

    m = X.shape[0]
    C = 0
    w1_grad = np.zeros((w1.shape))
    w2_grad = np.zeros((w2.shape))

    # Forward Step
    #########################################################################################
   






    #########################################################################################
    
    # ONE HOT ENCODING (y to Y)
    #########################################################################################
  





    #########################################################################################

    #Penalty
    #########################################################################################
    
    #########################################################################################
    
    # Cost Function
    #########################################################################################
    
    #########################################################################################

    # Back Propagation
    #########################################################################################
    
    
    
    
    
    
    
    
    
    
    
    
    
    #########################################################################################
    grad = np.append(w1_grad.flatten(), w2_grad.flatten())

    return C, grad

In [ ]:
Lambda = 0    
C, grad = nnCostFunction(nn_params,input_layer_size,hidden_layer_size,num_labels,X,y,Lambda)
print("Cost at debugging parameters: %.6f \nFor Lambda = 0 it should be:  0.287629" % C)
Lambda = 3    
C, grad = nnCostFunction(nn_params,input_layer_size,hidden_layer_size,num_labels,X,y,Lambda)
print("Cost at debugging parameters: %.6f \nFor Lambda = 3 it should be:  0.576051" % C)

### 5: Kezdeti súlyok

A kezdeti súlyokat egyenletes eloszlással inicializáljuk egy meghatározott intervallumon. A súlyok így nem 0 kezdeti értékről indulnak, ami lehetővé teszi a konvergenciát.  

In [ ]:
def randInitializeWeights(L_in, L_out):
    epsilon_init = 0.12
    W = np.random.rand(L_out,L_in+1)*(2*epsilon_init)-epsilon_init
    return W

In [ ]:
initial_w1 = randInitializeWeights(input_layer_size,hidden_layer_size)
initial_w2 = randInitializeWeights(hidden_layer_size,num_labels)
initial_nn_params = np.append(initial_w1.flatten(),initial_w2.flatten())

### 6: Gradiens módszer

A háló tanításához implementáljuk a gradiens módszert.

In [ ]:
def gradientDescentnn(X, y, initial_nn_params, lr_rate, num_iters, Lambda, input_layer_size, hidden_layer_size,
                      num_labels):

    w1 = initial_nn_params[:((input_layer_size + 1) * hidden_layer_size)].reshape(hidden_layer_size, input_layer_size + 1)
    w2 = initial_nn_params[((input_layer_size + 1) * hidden_layer_size):].reshape(num_labels, hidden_layer_size + 1)

    m = len(y)
    C_history = []

    for i in range(num_iters):
        if (i%20==0):
            print('Iteration:', i+1)
        elif (i==num_iters-1):
            print('Done!')
        
        nn_params = np.append(w1.flatten(), w2.flatten())
        C, grad = nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, Lambda)
        
        w1_grad = grad[:((input_layer_size+1) * hidden_layer_size)].reshape(hidden_layer_size,input_layer_size+1)
        w2_grad = grad[((input_layer_size +1)* hidden_layer_size ):].reshape(num_labels,hidden_layer_size+1)
        
        w1 = w1 - (lr_rate * w1_grad)
        w2 = w2 - (lr_rate * w2_grad)
        C_history.append(C)

    nn_paramsFinal = np.append(w1.flatten(), w2.flatten())
    return nn_paramsFinal, C_history

In [ ]:
# Tanulás paramétereinek beállítása
lr_rate = 0.9
num_iter = 400
Lambda = 0.1

nnw, nnC_history = gradientDescentnn(X,y,initial_nn_params,lr_rate,num_iter,Lambda,input_layer_size,hidden_layer_size,num_labels)
w1 = nnw[:((input_layer_size+1) * hidden_layer_size)].reshape(hidden_layer_size,input_layer_size+1)
w2 = nnw[((input_layer_size +1)* hidden_layer_size ):].reshape(num_labels,hidden_layer_size+1)

A futtatás paramétereivel (Lambda, tanulási ráta, iterációk száma) lehet kísérletezni.

Az adott tanítás sikerességének vizsgálatához implementáljunk egy prediktáló függvényt, ami végrehajtja az előre lépést (Forward Step) és eredményt eredeti alakra alakítja vissza.

In [ ]:
def predict(w1,w2,X):
    m = X.shape[0]
    #Forward step
    ########################################################
    
    
    
    
    
    
    
    
    ########################################################
    return p

A kevés mintaszám miatt a tanító adatokon ugyen, de vizsgáljuk meg a háló pontosságát, vagyis vizsgáljuk meg, hogy az összes minta közül hányat talált el megfelelően. Az adatok alapaján adjuk vissza a háló pontosságát százalékban.

In [ ]:
def accuracy(pred,y):
    acc = (np.mean(pred[:,np.newaxis]==y))*100
    return acc

In [ ]:
pred = predict(w1,w2,X)
acc = accuracy(pred,y)

print('Training Set Accuracy after %.0f iteration (currently trained weights): %.2f %%' % (num_iter,acc))

w1_r = np.loadtxt("w1_final.txt")
w2_r = np.loadtxt("w2_final.txt")

pred_800 = predict(w1_r,w2_r,X)
acc_800 = accuracy(pred_800,y)
print('Training Set Accuracy after 800 iteration (loaded weights): %.2f %%' % acc_800)

### 7: Tanulás lefutása
A költségfüggvény interációnkénti kirajzolásával vizsgáljuk meg a tanulás lefutását. 

In [ ]:
plt.plot(nnC_history)
plt.xlabel('Iteration')
plt.ylabel('Cost function')
plt.title('Cost function over the iterations (Current params)')
plt.show()

nnC_history_load=np.loadtxt("nnC_history_800.txt")
plt.plot(nnC_history_load)
plt.xlabel('Iteration')
plt.ylabel('Cost function')
plt.title('Cost function over the iterations (Loaded params)')
plt.show()

## Magasabb szintű csomagokkal

A Keras (vagy Tensorflow) csomagokkal a fentebb részletesen ismertettett eljárások egyszerűbben és tömörebb kódként implementálhatóak. A függvények optimalizáltak. Lássuk a fenti példa hogyan implementálható tömörebben.

In [ ]:
# imports for array-handling and plotting
import numpy as np
import matplotlib.pyplot as plt

# keras imports for the dataset and building our neural network
from keras.models import Sequential
from keras.layers.core import Dense, Activation

Modell létrehozása. Rétegek, neuron számok és aktivációs függvények definiálása.

In [ ]:
# building a linear stack of layers with the sequential model
model = Sequential()
model.add(Dense(400, input_shape=(400,)))
model.add(Activation('sigmoid'))                            

model.add(Dense(25))
model.add(Activation('sigmoid'))

model.add(Dense(10))
model.add(Activation('softmax'))

Költségfüggvény definiálása.

In [ ]:
# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

Az eredeti adatokból kiindulva itt is szükség van a One Hot átkódolásra.

In [ ]:
Y = np.zeros((m, num_labels))
I = np.eye(num_labels)

for i in range(1, m+1):
    Y[i-1, :] = I[y[i-1]-1, :]

Háló tanítása, metrikák kinyerése.

In [ ]:
# training the model and saving metrics in history
history = model.fit(X, Y, epochs=20, verbose = 2)

Tanítás lefutásának ellenőrzése a metrikák segítségével.

In [ ]:
# plotting the metrics
fig = plt.figure()
plt.plot(history.history['acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'])